In [1]:
from pathlib import Path
import numpy as np
from heapq import heappush, heappop
from itertools import groupby

In [2]:
compact = [int(a) for a in Path("data/9.txt").read_text().strip()]
disk = np.full(sum(compact), -1)
lensum = 0
for i in range(0, len(compact)):
    el = i // 2 if i % 2 == 0 else -1
    disk[lensum : lensum + compact[i]] = el
    lensum += compact[i]

In [3]:
empty = [x for x in np.where(disk == -1)[0]]
to_move = [x for x in np.where(disk != -1)[0][::-1]]
defrag = disk.copy()

while empty and to_move:
    empty_pos = heappop(empty)
    to_move_pos = to_move[0]
    if empty_pos >= to_move_pos:
        break
    defrag[empty_pos], defrag[to_move_pos] = defrag[to_move_pos], defrag[empty_pos]
    to_move = to_move[1:]
    heappush(empty, to_move_pos)

(np.where(defrag == -1, 0, defrag) * np.arange(0, len(defrag))).sum()

np.int64(6323641412437)

In [4]:
defrag = disk.copy()
contiguous = [
    (k, min(l := [x[1] for x in g]), max(l) + 1)
    for k, g in groupby(zip(defrag, range(len(defrag))), key=lambda x: x[0])
]
files = [x for x in contiguous if x[0] != -1][::-1]
spaces = [x for x in contiguous if x[0] == -1]

for file, file_start, file_end in files:
    for i, (space, space_start, space_end) in enumerate(spaces):
        if file_start <= space_start:
            break
        if file_end - file_start <= space_end - space_start:
            defrag[space_start : space_start + file_end - file_start] = file
            defrag[file_start:file_end] = -1
            spaces[i] = space, space_start + file_end - file_start, space_end
            break

(np.where(defrag == -1, 0, defrag) * np.arange(0, len(defrag))).sum()

np.int64(6351801932670)